In [ ]:

from searchalgorithms import *
from usermodel import User_model

from timeit import default_timer as timer
import networkx as nx
# np.random.seed(5)
# random.seed(5)
randint = random.randint
# https://github.com/jmschrei/pomegranate/blob/master/tutorials/B_Model_Tutorial_4b_Bayesian_Network_Structure_Learning.ipynb
# https://github.com/pgmpy/pgmpy
# another example: https://github.com/pgmpy/pgmpy_notebook/blob/master/notebooks/9.%20Learning%20Bayesian%20Networks%20from%20Data.ipynb

# from https://github.com/jmschrei/pomegranate/blob/master/examples/bayesnet_asia.ipynb

class ProblemDomain:
    """"This class generate an array of devices (available_devices)
    [0 0 1 1 2 2 3 3] where each type repeated by number of alternatives, the value
    represents the function they can do, so capabilities only one for now.
    e.g. device number 2,3 can do task number 1 """

    def __init__(self, n_dev_types,n_dev_alter, subtask_pool_list, n_dev_capab, task):

        self.subtask_pool_list = subtask_pool_list
        self.n_dev_types = n_dev_types
        self.n_dev_alt = n_dev_alter
        self.n_total_devices = n_dev_types * n_dev_alter
        self.n_capab = n_dev_capab
        self.task = task

        self.all_available_devices = self.gen_devices()
        self.subtask_dev = self._get_subtask_dev()

    def gen_devices(self):
        # return an array with a list of all devices
        # index is the device, value is the function it can do (capabilities).
        # e.g. [0 0 1 1 2 2 3 3], four type of devices each two have same capabilities
        # d0,d1 can do function 0, d4,d5 can do function 2
        return list(np.array([i//self.n_dev_alt for i in range(self.n_total_devices)]))

    def get_all_neighbors(self, cand):
        # Get all other cand that can do the task but differ from current cand by one device.
        neighbor_list = []
        for cand_idx in range(len(cand)):
            cand_dev_idx = cand[cand_idx]
            fnk = self.all_available_devices[cand_dev_idx]
            fnk_dev_idx = np.where(np.isin(self.all_available_devices,fnk))[0]
            for fnk_dev_idx_alt in fnk_dev_idx:
                if  fnk_dev_idx_alt != cand_dev_idx:
                    new_neghbor = cand[:]
                    new_neghbor[cand_idx] = fnk_dev_idx_alt
                    neighbor_list.append(new_neghbor)


        return neighbor_list


    # return all devices that can exectue this task
    def _get_subtask_dev(self):
        # return a dict: key is fun idx, value is a list of dev idx that are 
        # cabable to execute the func.
        self.subtask_dev = {}
        for f_idx in self.task:
            self.subtask_dev[f_idx] = []

        for d_idx in range(len(self.all_available_devices)):
            f_id = self.all_available_devices[d_idx]
            if f_id in self.task:
                self.subtask_dev[f_id].append(d_idx)


        # self.prob_domain_size =1
        # for f_id, dev_lst in self.subtask_dev.items():
        #     self.prob_domain_size *= len(dev_lst)

        return self.subtask_dev #, self.prob_domain_size

    def get_rand_solution(self):
        # return a random solution 
        # list of dev_idx ordered by task number (i.e. first dev for first task)
        sol = []
        for t in self.task:
            sol.append(random.choice(self.subtask_dev[t]))

        return sol

    def get_dev_instance(self, dev_id_lst):
        # this one should be similar to usermodel alter_list

        alt_list = [str(c) for c in range(self.n_dev_alt)]
        return {self.all_available_devices[dev_id]:alt_list[dev_id%self.n_dev_alt] \
            for dev_id in dev_id_lst}
        
def main(iteration):
    # number of devices type e.g. door locks, lights, coffee makers
    n_devices = 3
    # all functions in the IoT devices
    n_functions = 5
    # number of funcitons for each device
    n_funct_per_device = 5

    devices = { "d" + str(i) : [] for i in range(n_devices) }
    functions = [ "f" + str(i) for i in range(n_functions) ]
    for d in devices.keys():
        devices[d].extend( random.sample(functions, n_funct_per_device))
    
    func_alter_devices = {f: [] for f in functions}
    for d, df_list in devices.items():
        for f in df_list:
            func_alter_devices[f].append(d)
    
    user_model = User_model(nodes = functions, \
                            n_edges = randint(int(n_functions / 4), n_functions), \
                            devices = devices, \
                            func_alter_devices = func_alter_devices, \
                            is_gen_task = True)
    print(user_model.devices)
    print(user_model.nodes)
    print(user_model.func_alter_devices)
    print(user_model.task_dict)
    #print(user_model.child_parent)
    #nx.draw(user_model.DAG,with_labels = True)
    return 1

    best_cand = [t*n_dev_alter+int(alt) for t,alt in user_model.task_dict.items()]
    print("Task:best devices:", user_model.task_dict)
    print("best_cand:", best_cand, \
        " score: ", user_model.get_score( best_cand) )


    task = list(user_model.task_dict.keys())
    prob_domain = ProblemDomain(n_devices = n_devices, \
        n_funct_per_device = n_funct_per_device, \
        functions_pool_list = functions_pool_list, \
        task = task)

    start = timer()
    exh_search = BruteForceSearch(prob_domain.subtask_dev, \
        user_model.get_score).run()
    print("Brute Force Search: ", exh_search[0], " ", exh_search[1], \
        " instances> ", prob_domain.get_dev_instance(exh_search[1]))
    end = timer()

    sys.exit(0)
    with open('data.txt-test','a+') as f:
        f.write('{0} $ {1} $ {2} $ {3} $ {4} $ {5} $ {6} $ {7} $ {8} $ {9}' \
            .format(iteration, \
            user_model.task_dict.keys(), \
            len(user_model.task_dict.keys()), \
            best_cand,  \
            user_model.get_score( best_cand)[0], \
            end - start, \
            exh_search[0],  \
            exh_search[1], \
            prob_domain.get_dev_instance(exh_search[1]), \
            prob_domain.subtask_dev) + "\n")


    
    for i in range(30):
        print("internal iteration:", i)
        # define heuristic algorithms objects
        start = timer()
        hc = HillClimbing(prob_domain.get_all_neighbors, user_model.get_score)
        init_cand = prob_domain.get_rand_solution()
        (h_cand, h_score) = hc.climb(init_cand)
        end = timer()
        hc_time = end - start
        print("Elapse time for HC is {} sec.".format(end - start))
        
        start = timer()
        ga = GA(prob_domain, user_model.get_score)
        ga_result = ga.run(n=1000, max_iteration=1000)
        end = timer()
        ga_time = end - start
        print("Elapse time for GA is {} sec ".format(end - start))

        start = timer()
        simulated_annealing = TasktoIoTmapingProblem(init_cand, \
            prob_domain, user_model.get_score)
        (s_cand, s_score) = simulated_annealing.anneal()
        end = timer()
        sa_time = end - start
        print("Elapse time for SA is {} sec ".format(end - start))
        result = '{0} $ {1} $ {2} $ {3} $ {4} $ {5} $ {6} $ {7} $ {8} $ \
                {9} $ {10} $ {11} $ {12} $ {13}'.format( \
                iteration,  \
                i , \
                1.0 - s_score,  \
                h_score,  \
                ga_result[1][0],  \
                sa_time, \
                hc_time, \
                ga_time, \
                s_cand,  \
                h_cand,  \
                ga_result[0], \
                prob_domain.get_dev_instance(s_cand), \
                prob_domain.get_dev_instance(h_cand), \
                prob_domain.get_dev_instance(ga_result[0])) 
        print(result)
        with open('data-test.txt','a+') as f:
            f.write(result+"\n")             


if __name__ == "__main__":
    start = timer()
    for i in range(1):
        print("external iteration: ", i)
        main(i)
    end = timer()
    print("Over all Elapse time is sec {}".format(end-start))
